In [3]:

try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

In [4]:
import cirq
import numpy as np
import collections

In [5]:

# --- 1. Define a Simple Quantum Circuit ---
# This section defines a basic quantum circuit that we will later transpile
# and run with error mitigation.

def create_example_circuit(qubit: cirq.Qid) -> cirq.Circuit:
    """
    Creates a simple quantum circuit that prepares a superposition state
    and then measures it.
    """
    # Initialize an empty Cirq circuit.
    circuit = cirq.Circuit()
    # Apply a Hadamard gate to create a superposition.
    # H gate puts the qubit into (|0> + |1>)/sqrt(2) state.
    circuit.append(cirq.H(qubit))
    # Apply a Z gate (phase flip), which should not change measurement probabilities
    # in the computational basis if only H and measurement are present, but
    # serves as an example for gate decomposition.
    circuit.append(cirq.Z(qubit))
    # Measure the qubit. The result will be stored under the key 'result'.
    circuit.append(cirq.measure(qubit, key='result'))
    print("--- Original Circuit ---")
    print(circuit)
    print("-" * 30)
    return circuit


In [6]:

# --- 2. Circuit Transpilation (Optimization for Target Backend) ---
# Transpilation is the process of rewriting a quantum circuit to make it
# compatible with a specific quantum hardware device's capabilities (e.g.,
# its native gate set, connectivity, etc.) and to optimize its performance.

def transpile_circuit(circuit: cirq.Circuit) -> cirq.Circuit:
    """
    Transpiles the given circuit for a hypothetical target backend.
    For demonstration, we'll use cirq.SqrtIswapTargetGateset, which represents
    a common gate set for superconducting quantum computers.
    This process involves decomposing gates into the target gateset and
    applying various optimizations (e.g., gate cancellations, merging).
    """
    print("--- Transpiling Circuit for Target Gateset ---")
    # Define the target gateset. cirq.SqrtIswapTargetGateset includes:
    # - cirq.PhasedXPowGate (single-qubit rotations)
    # - cirq.ISwapPowGate (two-qubit entangling gate, specifically sqrt(iSWAP))
    # - cirq.MeasurementGate
    # This function will attempt to rewrite all operations in the circuit
    # using only gates from this target gateset.
    # It also applies various built-in transformers for optimization.
    optimized_circuit = cirq.optimize_for_target_gateset(
        circuit,
        gateset=cirq.SqrtIswapTargetGateset()
    )
    print("--- Transpiled Circuit ---")
    print(optimized_circuit)
    print("-" * 30)
    return optimized_circuit

In [7]:


# --- 3. Simulate Noisy Measurements for Error Mitigation ---
# To demonstrate error mitigation, we first need to simulate a noisy environment.
# Here, we'll simulate simple bit-flip errors during measurement.
# In a real scenario, this noise would come from the quantum hardware itself.

def simulate_noisy_measurement(
    circuit: cirq.Circuit,
    repetitions: int,
    bit_flip_probability: float = 0.05
) -> collections.Counter:
    """
    Simulates running the circuit with a noisy measurement channel.
    A bit-flip error is applied to the measurement outcome with a given probability.
    """
    print(f"--- Simulating Noisy Circuit with {bit_flip_probability*100}% Bit-Flip Probability ---")
    simulator = cirq.Simulator()
    # Run the circuit without explicit noise models in the simulator,
    # but apply a classical bit-flip post-processing to simulate measurement noise.
    # This is a simplified approach for demonstration.
    raw_results = simulator.run(circuit, repetitions=repetitions)

    noisy_counts = collections.Counter()
    for _ in range(repetitions):
        # Get the ideal measurement result (before applying simulated noise)
        # Assuming a single measurement key 'result' and a single qubit.
        ideal_measurement = raw_results.measurements['result'][_][0]

        # Simulate a bit-flip error
        if np.random.rand() < bit_flip_probability:
            noisy_measurement = 1 - ideal_measurement  # Flip 0 to 1, or 1 to 0
        else:
            noisy_measurement = ideal_measurement

        noisy_counts[noisy_measurement] += 1

    print("Noisy Measurement Counts:", noisy_counts)
    print("-" * 30)
    return noisy_counts

In [8]:


# --- 4. Measurement Error Mitigation ---
# Measurement error mitigation aims to correct for errors that occur during
# the final readout of the qubits. This is typically done by characterizing
# the measurement noise and then applying a classical post-processing step.

def calibrate_measurement_noise(
    qubit: cirq.Qid,
    repetitions: int,
    bit_flip_probability: float = 0.05
) -> np.ndarray:
    """
    Calibrates the measurement noise by running simple circuits to
    determine the probability of measuring 0 when the qubit should be 1,
    and measuring 1 when the qubit should be 0.
    Returns a confusion matrix (2x2) for the measurement channel.
    Matrix M:
    M[i, j] = probability of measuring j given ideal state i
    M = [[P(meas 0 | ideal 0), P(meas 1 | ideal 0)],
         [P(meas 0 | ideal 1), P(meas 1 | ideal 1)]]
    """
    print("--- Calibrating Measurement Noise ---")
    simulator = cirq.Simulator()

    # Circuit to prepare |0> and measure
    circuit_0 = cirq.Circuit(cirq.measure(qubit, key='result'))
    # Circuit to prepare |1> and measure (apply X gate to |0>)
    circuit_1 = cirq.Circuit(cirq.X(qubit), cirq.measure(qubit, key='result'))

    # Simulate noisy measurements for |0> state
    counts_0_ideal = simulator.run(circuit_0, repetitions=repetitions)
    noisy_counts_0 = collections.Counter()
    for _ in range(repetitions):
        ideal_measurement = counts_0_ideal.measurements['result'][_][0]
        if np.random.rand() < bit_flip_probability:
            noisy_measurement = 1 - ideal_measurement
        else:
            noisy_measurement = ideal_measurement
        noisy_counts_0[noisy_measurement] += 1

    # Simulate noisy measurements for |1> state
    counts_1_ideal = simulator.run(circuit_1, repetitions=repetitions)
    noisy_counts_1 = collections.Counter()
    for _ in range(repetitions):
        ideal_measurement = counts_1_ideal.measurements['result'][_][0]
        if np.random.rand() < bit_flip_probability:
            noisy_measurement = 1 - ideal_measurement
        else:
            noisy_measurement = ideal_measurement
        noisy_counts_1[noisy_measurement] += 1

    # Calculate probabilities for the confusion matrix
    # P(meas 0 | ideal 0)
    p_00 = noisy_counts_0.get(0, 0) / repetitions
    # P(meas 1 | ideal 0)
    p_01 = noisy_counts_0.get(1, 0) / repetitions
    # P(meas 0 | ideal 1)
    p_10 = noisy_counts_1.get(0, 0) / repetitions
    # P(meas 1 | ideal 1)
    p_11 = noisy_counts_1.get(1, 0) / repetitions

    # Construct the confusion matrix
    # M[i, j] = P(measured j | ideal i)
    # Row 0: Ideal 0, Measured 0, Measured 1
    # Row 1: Ideal 1, Measured 0, Measured 1
    confusion_matrix = np.array([[p_00, p_01],
                                 [p_10, p_11]])

    print("Measurement Confusion Matrix (P(measured j | ideal i)):")
    print(confusion_matrix)
    print("-" * 30)
    return confusion_matrix

In [9]:


def apply_measurement_error_mitigation(
    noisy_counts: collections.Counter,
    confusion_matrix: np.ndarray
) -> np.ndarray:
    """
    Applies classical measurement error mitigation using the inverse of the
    confusion matrix. This technique attempts to infer the true probabilities
    from the noisy observed probabilities.

    Args:
        noisy_counts: A Counter object with observed noisy measurement outcomes.
                      Assumes single qubit measurement (0 or 1).
        confusion_matrix: A 2x2 numpy array representing the measurement
                          confusion matrix.

    Returns:
        A numpy array of mitigated probabilities for [0, 1].
    """
    print("--- Applying Measurement Error Mitigation ---")

    # Observed noisy probability vector [P(meas 0), P(meas 1)]
    total_shots = sum(noisy_counts.values())
    observed_probs = np.array([noisy_counts.get(0, 0) / total_shots,
                               noisy_counts.get(1, 0) / total_shots])

    print("Observed Noisy Probabilities:", observed_probs)

    # Invert the confusion matrix to get the correction matrix
    # If M is the confusion matrix, and P_noisy = P_ideal @ M,
    # then P_ideal = P_noisy @ M_inv (where @ is matrix multiplication)
    try:
        correction_matrix = np.linalg.inv(confusion_matrix)
    except np.linalg.LinAlgError:
        print("Warning: Could not invert confusion matrix. Results may be unreliable.")
        return observed_probs # Return unmitigated if inversion fails

    # Apply the correction
    mitigated_probs = np.dot(observed_probs, correction_matrix)

    # Normalize and clip to ensure valid probabilities (between 0 and 1)
    mitigated_probs = np.clip(mitigated_probs, 0, 1)
    mitigated_probs /= np.sum(mitigated_probs) # Re-normalize after clipping

    print("Mitigated Probabilities:", mitigated_probs)
    print("-" * 30)
    return mitigated_probs


In [16]:


# --- Main Execution ---
if __name__ == "__main__":
    # Define a qubit for our circuit
    q = cirq.LineQubit(0)
    # Number of repetitions for simulation
    num_repetitions = 10000
    # Simulated bit-flip probability for demonstration
    simulated_bit_flip_prob = 0.1

    # Step 1: Create the original circuit
    original_circuit = create_example_circuit(q)

    # Step 2: Transpile the circuit
    transpiled_circuit = transpile_circuit(original_circuit)

    # Step 3: Simulate noisy measurements on the transpiled circuit
    # This represents running the circuit on a real, noisy device.
    noisy_measurement_counts = simulate_noisy_measurement(
        transpiled_circuit,
        num_repetitions,
        simulated_bit_flip_prob
    )

    # Step 4a: Calibrate the measurement noise
    # This step would typically be done once for a given device and noise model.
    confusion_matrix = calibrate_measurement_noise(
        q,
        num_repetitions,
        simulated_bit_flip_prob
    )

    # Step 4b: Apply measurement error mitigation
    mitigated_probabilities = apply_measurement_error_mitigation(
        noisy_measurement_counts,
        confusion_matrix
    )

    # --- Compare Results ---
    print("\n--- Summary of Results ---")
    # The ideal outcome for the original circuit (H then Z then M) should be
    # approximately 50% for 0 and 50% for 1.
    print(f"Ideal Expected Probabilities: [0.5, 0.5]")
    # Convert noisy counts to probabilities for comparison
    total_noisy_shots = sum(noisy_measurement_counts.values())
    noisy_probabilities = np.array([noisy_measurement_counts.get(0, 0) / total_noisy_shots,
                                    noisy_measurement_counts.get(1, 0) / total_noisy_shots])
    print(f"Observed Noisy Probabilities: {noisy_probabilities}")
    print(f"Mitigated Probabilities:    {mitigated_probabilities}")

    # Calculate the error (difference from ideal) for noisy vs. mitigated results
    ideal_probs = np.array([0.5, 0.5])
    noisy_error = np.linalg.norm(noisy_probabilities - ideal_probs)
    mitigated_error = np.linalg.norm(mitigated_probabilities - ideal_probs)

    print(f"\nError (L2 norm) from Ideal for Noisy:     {noisy_error:.4f}")
    print(f"Error (L2 norm) from Ideal for Mitigated: {mitigated_error:.4f}")

    if mitigated_error < noisy_error:
        print("\nMeasurement error mitigation successfully reduced the error!")
    else:
        print("\nMeasurement error mitigation did not improve results (or made them worse).")
        print("This can happen with high noise levels or insufficient repetitions for calibration.")

--- Original Circuit ---
0: ───H───Z───M('result')───
------------------------------
--- Transpiling Circuit for Target Gateset ---
--- Transpiled Circuit ---
0: ───PhXZ(a=0.5,x=-0.5,z=0)───M('result')───
------------------------------
--- Simulating Noisy Circuit with 10.0% Bit-Flip Probability ---
Noisy Measurement Counts: Counter({np.int8(1): 5022, np.int8(0): 4978})
------------------------------
--- Calibrating Measurement Noise ---
Measurement Confusion Matrix (P(measured j | ideal i)):
[[0.9037 0.0963]
 [0.101  0.899 ]]
------------------------------
--- Applying Measurement Error Mitigation ---
Observed Noisy Probabilities: [0.4978 0.5022]
Mitigated Probabilities: [0.49433163 0.50566837]
------------------------------

--- Summary of Results ---
Ideal Expected Probabilities: [0.5, 0.5]
Observed Noisy Probabilities: [0.4978 0.5022]
Mitigated Probabilities:    [0.49433163 0.50566837]

Error (L2 norm) from Ideal for Noisy:     0.0031
Error (L2 norm) from Ideal for Mitigated: 0.008